# Trabalho Final de Inteligência Artificial
### Aluno: Paulo Henrique Dionysio | RA: 221026169

### Previsão de preços de casas usando redes neurais

## Importação de Bibliotecas Necessárias

In [ ]:
# Importação das bibliotecas necessárias
import streamlit as st  # Para construção da interface interativa
import pandas as pd  # Manipulação de dados tabulares
import numpy as np  # Operações numéricas
import tensorflow as tf  # Construção e treinamento de redes neurais
from sklearn.preprocessing import MaxAbsScaler  # Normalização de dados
from sklearn.impute import SimpleImputer  # Tratamento de valores ausentes
from sklearn.model_selection import train_test_split  # Separação dos dados em treino e validação
import plotly.express as px  # Visualização de gráficos interativos


## Configuração Inicial da Página

In [1]:
# Configuração inicial da página
st.title("Previsão de Preços de Casas")
st.subheader("Trabalho Final de Inteligência Artificial")
st.write("Aluno: Paulo Henrique Dionysio | RA: 221026169")

NameError: name 'st' is not defined

## Função para Carregar Dados

In [ ]:
# Função para carregar e concatenar os dados de treino e teste
dados_treino = pd.read_csv('test.csv')
dados_teste = pd.read_csv('train.csv')
dados = pd.concat([dados_treino, dados_teste], sort=False)

## Função de Pré-Processamento

In [ ]:
# Função para realizar o pré-processamento dos dados
def preprocessar_dados(dados, dados_treino, scaler, imputer):
    """
    Pré-processa os dados de entrada, incluindo normalização e imputação.
    Args:
        dados: Dados combinados de treino e teste.
        dados_treino: Dados originais de treino.
        scaler: Objeto de normalização.
        imputer: Objeto para tratar valores ausentes.
    Returns:
        Dados pré-processados para treino, validação e teste.
    """
    # Transformação de variáveis categóricas em dummies
    cat_cols = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
                'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
                'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
                'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
                'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
                'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
                'GarageCond', 'PavedDrive', 'PoolQC', 'MiscFeature', 'SaleType',
                'SaleCondition', 'Fence']
    juntos_num = pd.get_dummies(dados, columns=cat_cols)
    
    # Separação dos dados em treino e teste
    treino_num = juntos_num.iloc[:len(dados_treino), :]
    teste_num = juntos_num.iloc[len(dados_treino):, :]

    X = treino_num.drop(["Id", "SalePrice"], axis=1)
    y = treino_num["SalePrice"]
    
    # Divisão dos dados de treino em treino e validação
    Xtr, Xval, ytr, yval = train_test_split(X, y, train_size=0.5, random_state=0)

    # Normalização e imputação
    Xtr = scaler.fit_transform(imputer.fit_transform(Xtr))
    Xval = scaler.transform(imputer.transform(Xval))
    ytr = ytr.values
    yval = yval.values

    return Xtr, Xval, ytr, yval, treino_num, teste_num


## Pré-Processamento e Visualização

In [ ]:
if treino_file and teste_file:
    # Carregamento e visualização dos dados
    dados_treino, dados_teste, dados = carregar_dados(treino_file, teste_file)

    scaler = MaxAbsScaler()
    imputer = SimpleImputer(strategy="median")
    Xtr, Xval, ytr, yval, treino_num, teste_num = preprocessar_dados(
        dados, dados_treino, scaler, imputer
    )

    st.write("Amostra dos Dados de Treino:")
    st.write(dados_treino.head())

## Treinamento do Modelo

In [5]:
# Dividindo em treino e teste apenas com variáveis numéricas
treino_num = juntos_num.iloc[0:1460, :]
teste_num = juntos_num.iloc[1460:2920, :]

In [6]:
# Separando Variável X e y
X = treino_num.drop(["Id","SalePrice"], axis=1)
y = treino_num['SalePrice']

In [7]:
# Colando os valores de teste no formato desejado
teste1 = teste_num.drop(["Id", "SalePrice"], axis=1)

In [8]:
# Dividindo em treino e validação
Xtr, Xval, ytr, yval = train_test_split(X,y, train_size=0.5, random_state=0)

Xtr.shape, Xval.shape, ytr.shape, yval.shape

((730, 287), (730, 287), (730,), (730,))

In [9]:
# Aplicando a mediana aos valores faltantes aos dados de treino, validação e teste
imputer = SimpleImputer(strategy='median')
Xtr = imputer.fit_transform(Xtr)
Xval = imputer.transform(Xval)

teste = imputer.fit_transform(teste1)

In [10]:
# Aplicando a normalização aos dados de treino, validação e teste
scaler = MaxAbsScaler()
Xtr = scaler.fit_transform(Xtr)
Xval = scaler.transform(Xval)

teste = scaler.fit_transform(teste)

In [11]:
# Transformando para numpy array
ytr = ytr.values
yval = yval.values

In [12]:
# Montando a arquitetura da rede neural
tf.random.set_seed(2)

inp = tf.keras.layers.Input((Xtr.shape[1], ))

hid1 = tf.keras.layers.Dense(100, activation='relu')(inp)

drop = tf.keras.layers.Dropout(0.5)(hid1)

hid2 = tf.keras.layers.Dense(50,activation='relu')(drop)

hid3 = tf.keras.layers.Dense(25, activation='relu')(hid2)

out = tf.keras.layers.Dense(1,activation='relu')(hid3)

mdl = tf.keras.Model(inp, out)
mdl.compile(loss='msle', optimizer='adam')
mdl.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 287)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        28,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │         1,275 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,151 (137.31 KB)

 Trainable params: 35,151 (137.31 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Criando um método para que a rede pare quando tenha convergido
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.1, patience=10, mode='min', restore_best_weights=True
)

In [14]:
# Treinando o modelo
mdl.fit(Xtr, ytr, validation_data=(Xval, yval), epochs=100, shuffle=True, batch_size=1, callbacks=[es])

Epoch 1/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 82.8692 - val_loss: 8.9908
Epoch 2/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.2353 - val_loss: 1.7564
Epoch 3/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.4024 - val_loss: 0.4089
Epoch 4/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.3800 - val_loss: 0.1472
Epoch 5/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1964 - val_loss: 0.1088
Epoch 6/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1580 - val_loss: 0.1025
Epoch 7/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1649 - val_loss: 0.0957
Epoch 8/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1470 - val_loss: 0.0872
Epoch 9/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1376 - val_loss: 0.0756
Epoch 10/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1011 - val_loss: 0.0675
Epoch 11/100
730/730 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1123 - val_loss: 0.0573
Epoch 12/100
730/730 ━━━━━━━━━━━━━━━━━━━

In [15]:
# Prevendo com os dados de validação
from sklearn.metrics import mean_squared_log_error


p = mdl.predict(Xval)

# Calculando o erro
np.sqrt(mean_squared_log_error(yval, p))

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


np.float64(0.21208138110344926)

In [16]:
# Prevendo com os dados de teste o preço das casas
p = mdl.predict(teste)

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step


In [17]:
# Preços das casas
print("Preços das casas", p)


Preços das casas [[129595.8  ]
 [158440.64 ]
 [194848.9  ]
 ...
 [156465.05 ]
 [129881.914]
 [194752.55 ]]


In [18]:
# queremos remover entradas unidimensionais da forma de um array
p = p.squeeze()

In [19]:
# Formatando para ter o formato do kaggle, com o Id e a previsão do preço das casas
sub = pd.Series(p, index= teste_num['Id'], name='SalePrice')
